In [75]:
import pandas as pd
import datetime

In [76]:
# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
# Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
# Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

data = pd.read_csv('ml-latest-small/ratings.csv')
grouped = data.groupby('userId').agg({'rating':'count','timestamp':['min','max']})
grouped.loc[grouped.rating['count']>100,'lifetime'] = grouped.timestamp['max'] - grouped.timestamp['min']
str(datetime.timedelta(seconds=grouped.lifetime.mean()))

'463 days, 21:28:27.449612'

In [62]:
# Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# формально говоря в данном случае можно обойтись без группировки и суммирования, так как для каждого типа была  
# только одна перевозка, но реализовал более общий случай

# без указания адреса клиента
(rzd.groupby('client_id').agg({'rzd_revenue':'sum'})
     .join(auto.groupby('client_id').agg({'auto_revenue':'sum'}), how='outer')
     .join(air.groupby('client_id').agg({'air_revenue':'sum'}), how='outer')
)

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [59]:
# с указанием адреса клиента

(client_base
     .merge(rzd.groupby('client_id').agg({'rzd_revenue':'sum'}),on='client_id',how='left')
     .merge(auto.groupby('client_id').agg({'auto_revenue':'sum'}),on='client_id',how='left')
     .merge(air.groupby('client_id').agg({'air_revenue':'sum'}),on='client_id',how='left')
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [ ]:
# В задаче сквозной аналитики вам предоставили данные по местоположению пользователей.
# Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению 
# для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?
# Для составления ответа можно использовать вопросы:
    
# 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего
# потребуется одно-два числа. Как их получить?

# 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.
# Как это отразится на вопросе расчетах пункта 1?

# 3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


# Для добавления координат в таблицу визитов можно объединить её с таблицой с координатами по user_id, добавив дополнительный
# столбец с разницей между временем визита и временем снятия координаты.
# Минимальное значение этой дельты позволит с определенной точностью сопоставить визиту/покупке координаты.
# Завязка за время уменьшает значение ошибки из пункта два. Из координат можно получить город/страну/часовой пояс/язык, 
# это дает возможность принять решения, например, по локализации сайта, либо внедрение новых валют,
# либо увеличения кол-ва рекламы на конкретный регион.